In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
from datetime import datetime, timedelta

**Задача 1. Отличия до эксперимента**
  

Возьмите те же группы, что и в эксперименте с изменением дизайна сайта, и проверьте значимость отличий средней выручки с пользователя на неделе перед экспериментом (c 2022.03.16 по 2022.03.23).

Для решения используйте данные из файлов 2022-04-01T12_df_sales.csv и experiment_users.csv.

В качестве ответа введите p-value, округлённое до 3-го знака после точки.

In [2]:
df_sales = pd.read_csv('2022-04-01T12_df_sales.csv')
df_users = pd.read_csv('experiment_users.csv')

In [3]:
print(df_sales.shape)
display(df_sales.sample(3))
print(df_sales.info())

print(df_users.shape)
display(df_users.sample(3))
print(df_users.info())

(203847, 6)


,sale_id,date,count_pizza,count_drink,price,user_id
98793,1098794,2022-03-03 12:12:01,1,0,600,656f8d
145608,1145609,2022-03-15 20:49:50,1,1,780,dad1e8
106954,1106955,2022-03-05 13:48:03,3,0,1980,99f2f8


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 203847 entries, 0 to 203846
Data columns (total 6 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   sale_id      203847 non-null  int64 
 1   date         203847 non-null  object
 2   count_pizza  203847 non-null  int64 
 3   count_drink  203847 non-null  int64 
 4   price        203847 non-null  int64 
 5   user_id      203847 non-null  object
dtypes: int64(4), object(2)
memory usage: 9.3+ MB
None
(23333, 2)


,user_id,pilot
883,a57e7e,0
20713,2e9777,1
5019,6e3c16,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23333 entries, 0 to 23332
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   user_id  23333 non-null  object
 1   pilot    23333 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 364.7+ KB
None


In [4]:
df_sales['date'] = pd.to_datetime(df_sales['date'])

In [5]:
# количество пользователей в группах
df_users['pilot'].value_counts()

pilot
0    11769
1    11564
Name: count, dtype: int64

In [6]:
start_dt = datetime(2022, 3, 16)
end_dt = datetime(2022, 3, 23)

In [7]:
df_metric = df_sales[(df_sales['date']>=start_dt)&(df_sales['date']<end_dt)]\
    .groupby('user_id').agg(revenue = ('price', 'sum')).reset_index()

In [8]:
print(df_metric.shape)
display(df_metric.sample(5))

(24600, 2)


,user_id,revenue
9344,60fcef,780
15133,9c824a,600
13965,90f72c,2100
17798,b8699a,1320
20289,d2a536,600


In [9]:
df = df_users.merge(df_metric, how='left', on=['user_id'])
df['revenue'] = df['revenue'].fillna(0)

In [10]:
#Разделяем выборки на контроль и тест
df_control = df[df['pilot']==0]['revenue']
df_test = df[df['pilot']==1]['revenue']

stats.ttest_ind(df_control, df_test)

TtestResult(statistic=-1.2837567415000515, pvalue=0.19923983306424942, df=23331.0)

In [11]:
print('pvalue = ', round(stats.ttest_ind(df_control, df_test)[1], 3))

pvalue =  0.199


**Задача 2. Среднее время между покупками**
  

При выполнении прошлого задания вы могли заметить, что в данных много нулей. Это значит, что большая часть пользователей, совершивших покупку во время эксперимента, не совершала покупок на неделе до эксперимента. Интересно, как часто наши клиенты делают покупки?

Оцените среднее время между покупками. Возьмите всех клиентов, у которых 2 и более покупок. Вычислите время между покупками (для клиента с N покупками должно получиться N-1 значения времени). Объедините значения всех клиентов и вычислите среднее.

Для решения используйте данные из файлов `2022-04-01T12_df_sales.csv.`

В качестве ответа введите среднее количество дней между покупками, округлённое до целого значения.

In [12]:
print(df_sales.shape)
display(df_sales.sample(3))
print(df_sales.info())

(203847, 6)


,sale_id,date,count_pizza,count_drink,price,user_id
67536,1067537,2022-02-22 13:58:12,3,2,2130,9ec757
96674,1096675,2022-03-02 15:49:03,1,0,720,668988
28472,1028473,2022-02-11 17:59:52,1,0,720,f4e345


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 203847 entries, 0 to 203846
Data columns (total 6 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   sale_id      203847 non-null  int64         
 1   date         203847 non-null  datetime64[ns]
 2   count_pizza  203847 non-null  int64         
 3   count_drink  203847 non-null  int64         
 4   price        203847 non-null  int64         
 5   user_id      203847 non-null  object        
dtypes: datetime64[ns](1), int64(4), object(1)
memory usage: 9.3+ MB
None


In [13]:
#считаем кол-во заказов по каждому пользователю
cnt_orders_df = df_sales.groupby('user_id').agg(cnt_orders=('count_pizza', 'count')).reset_index()
#фильтруем пользователей с покупками >=2
cnt_orders_df = cnt_orders_df.loc[cnt_orders_df['cnt_orders']>=2]
#Отфильтровываем пользователей из df_sales
df_sales_filt = df_sales[df_sales['user_id'].isin(cnt_orders_df['user_id'].unique())]

In [14]:
df_sales_filt = df_sales_filt.sort_values(by=['user_id', 'date'])
#добавляем новый столбец с датой следующей покупки
df_sales_filt['next_date'] = df_sales_filt.groupby('user_id')['date'].shift(-1)

In [15]:
print(f"Среднее время между покупками: {round(((df_sales_filt['next_date']-df_sales_filt['date']).dt.total_seconds()/86400).mean(), )} дней")


Среднее время между покупками: 17 дней


**Задача 3. Функция для проверки статистической значимости**
 

Для проверки статистической значимости нужно вычислить p-value и сравнить его с уровнем значимости.

Напишите функцию `get_ttest_pvalue`

In [19]:
#Решение
def get_ttest_pvalue(metrics_a_group, metrics_b_group):
    """Применяет тест Стьюдента, возвращает pvalue.

    :param metrics_a_group (np.array): массив значений метрик группы A
    :param metrics_a_group (np.array): массив значений метрик группы B
    :return (float): значение p-value
    """
    pvalue = stats.ttest_ind(metrics_a_group, metrics_b_group)[1]

    return pvalue

In [20]:
metrics_a_group = np.array([964, 1123, 962, 1213, 914, 906, 951, 1033, 987, 1082])
metrics_b_group = np.array([952, 1064, 1091, 1079, 1158, 921, 1161, 1064, 819, 1065])
pvalue = get_ttest_pvalue(metrics_a_group, metrics_b_group)
print(pvalue)
# pvalue = 0.6122191629541949

0.6122191629541949
